# Capstone Project: Word2Vec for Recommendation Systems

> Generating movie recommendations using word2vec

The project documentation is deployed on: https://agacera.github.io/udacity_mle_word2vec_recommender/

# Running locally

This project uses conda to manage its dependencies, so make sure you have it installed.

To create an environment, run: `make conda-env`.
You can them activate this environment by running `conda activate word2vec_recommender`

# Development

To develop this project I used the library nbdev from Fast.ai, this library allows all the development to be taken inside Jupyter Notebooks, and in return it generates a beautiful documentation and exports the code from your notebook to Python modules. 

The main advantage of using it is that it allows a full exploratory development. So please, refer to the notebooks directly for code example and exploratory analysis.

# Dataset

This project uses the Movielens dataset. The data needed is already included in this repository using under `./data/`
